### Importing libraries

In [2]:
import numpy as np
import pandas as pd
from tensorflow.python.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.models import Sequential

### Load NSL-KDD dataset


In [2]:
url = "http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz"
column_names = ["duration", "protocol_type", "service", "flag", "src_bytes",
                "dst_bytes", "land", "wrong_fragment", "urgent", "hot",
                "num_failed_logins", "logged_in", "num_compromised", "root_shell",
                "su_attempted", "num_root", "num_file_creations", "num_shells",
                "num_access_files", "num_outbound_cmds", "is_host_login",
                "is_guest_login", "count", "srv_count", "serror_rate",
                "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
                "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
                "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate",
                "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
                "dst_host_srv_diff_host_rate", "dst_host_serror_rate",
                "dst_host_srv_serror_rate", "dst_host_rerror_rate",
                "dst_host_srv_rerror_rate", "label"]
df = pd.read_csv(url, names=column_names)
df.head()

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading nslkdd.zip to .\nslkdd


100%|██████████| 13.9M/13.9M [00:12<00:00, 1.15MB/s]


### Drop non-numeric columns and label column

In [ ]:
# Drop non-numeric columns and label column
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
df_numeric = df[numeric_columns]

### Data-Processing

In [ ]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(df_numeric)

# Split the data into training and test sets
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

### Define the autoencoder model

In [ ]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(32, activation='relu'),
    Dense(X_train.shape[1],activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

### Train the autoencoder

In [ ]:
model.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test))

### Evaluating the model

In [ ]:
# Use the trained autoencoder to encode and decode the test data
encoded_data = model.predict(X_test)

# Evaluate the reconstruction error
mse = np.mean(np.square(X_test - encoded_data))
print("Mean Squared Error:", mse)

### Function to detect anomalies

In [ ]:
# Function to detect anomalies
def detect_anomalies(X_test, threshold):
    reconstructed = model.predict(X_test)
    reconstruction_errors = np.mean(np.power(X_test - reconstructed, 2), axis=1)
    anomaly_indices = np.where(reconstruction_errors > threshold)[0]
    return anomaly_indices, reconstruction_errors

In [ ]:
threshold = 0.3  
anomaly_indices, reconstruction_errors = detect_anomalies(X_test, threshold)
print("Anomaly indices:", anomaly_indices)
print("Reconstruction errors:", reconstruction_errors)